In [6]:
import pandas as pd

# === Load and parse observed data ===
df = pd.read_csv("/home/mpasillaspablo/CIMIS/70_Manteca_hly2020.csv", dtype=str)

# -- Combine and clean 'Date' + 'Hour' columns --
date_strs = df["Date"].str.strip()
hour_strs = df["Hour (PST)"].str.zfill(4).str.replace(r"(\d{2})(\d{2})", r"\1:\2", regex=True)
dt_local = pd.to_datetime(date_strs + " " + hour_strs, format="%m/%d/%Y %H:%M", errors="coerce")
df["DateTime"] = dt_local.dt.tz_localize("America/Los_Angeles", ambiguous="NaT", nonexistent="NaT").dt.tz_convert("UTC").dt.tz_localize(None)

# -- Extract and clean the variable from column index 16 --
raw_var = df.iloc[:, 16].str.extract(r"(\d{1,3})")[0]
df["Variable"] = pd.to_numeric(raw_var, errors="coerce")

# -- Drop rows without valid DateTime or Variable --
df = df.dropna(subset=["DateTime", "Variable"])

# -- Filter for June 2020 only --
june_data = df[(df["DateTime"] >= "2020-06-01") & (df["DateTime"] < "2020-07-01")].copy()

# -- Set time index and ensure hourly regularity --
june_data = june_data.set_index("DateTime")[["Variable"]]  # Keep only numeric column
june_data = june_data.resample("h").asfreq()  # hourly frequency, no aggregation

# -- Interpolate missing values --
june_data["Variable"] = june_data["Variable"].interpolate(method="time", limit_direction="both")

# -- Remove the last 2 hours of June --
june_data = june_data.iloc[:-2]

# -- Final check for row count --
assert len(june_data) == 718, f"Unexpected row count: {len(june_data)}"

# -- Print result --
print(june_data["Variable"].to_list())


[53.0, 57.0, 58.0, 59.0, 62.0, 65.0, 68.0, 70.0, 72.0, 71.0, 74.0, 75.0, 79.0, 81.0, 74.0, 64.0, 59.0, 53.0, 46.0, 40.0, 39.0, 34.0, 32.0, 32.0, 31.0, 31.0, 34.0, 41.0, 55.0, 64.0, 66.0, 67.0, 68.0, 73.0, 82.0, 85.0, 82.0, 84.0, 66.0, 58.0, 53.0, 48.0, 43.0, 37.0, 35.0, 34.0, 32.0, 31.0, 31.0, 33.0, 38.0, 42.0, 47.0, 50.0, 46.0, 49.5, 53.0, 71.0, 80.0, 79.0, 82.0, 81.0, 59.0, 46.0, 44.0, 42.0, 41.0, 35.0, 29.0, 26.0, 24.0, 24.0, 29.0, 30.0, 33.0, 37.0, 41.0, 47.0, 42.0, 49.0, 56.0, 66.0, 77.0, 84.0, 90.0, 86.0, 61.0, 52.0, 46.0, 46.0, 41.0, 34.0, 30.0, 23.0, 22.0, 25.0, 28.0, 32.0, 29.0, 31.0, 33.0, 41.0, 46.0, 49.0, 52.0, 60.0, 65.0, 64.0, 66.0, 69.0, 65.0, 57.0, 53.0, 42.0, 38.0, 34.0, 33.0, 35.0, 37.0, 38.0, 37.0, 42.0, 51.0, 60.0, 65.0, 63.0, 62.0, 65.0, 68.0, 69.0, 71.0, 73.0, 74.0, 72.0, 65.0, 59.0, 53.0, 48.0, 44.0, 41.0, 37.0, 37.0, 35.0, 36.0, 38.0, 41.0, 48.0, 56.0, 61.0, 64.0, 67.0, 68.0, 69.0, 69.0, 71.0, 71.0, 72.0, 69.0, 61.0, 55.0, 49.0, 43.0, 39.0, 38.0, 36.0, 31.0, 30.

In [7]:
import pandas as pd

# === Load and parse observed data ===
df = pd.read_csv("/home/mpasillaspablo/CIMIS/70_Manteca_hly2020.csv", dtype=str)

# -- Combine and clean 'Date' + 'Hour' columns --
date_strs = df["Date"].str.strip()
hour_strs = df["Hour (PST)"].str.zfill(4).str.replace(r"(\d{2})(\d{2})", r"\1:\2", regex=True)
dt_local = pd.to_datetime(date_strs + " " + hour_strs, format="%m/%d/%Y %H:%M", errors="coerce")
df["DateTime"] = dt_local.dt.tz_localize("America/Los_Angeles", ambiguous="NaT", nonexistent="NaT").dt.tz_convert("UTC").dt.tz_localize(None)

# -- Extract and clean the variable from column index 16 --
raw_var = df.iloc[:, 16].str.extract(r"(\d{1,3})")[0]
df["RH (%)"] = pd.to_numeric(raw_var, errors="coerce")

# -- Drop rows without valid DateTime or RH values --
df = df.dropna(subset=["DateTime", "RH (%)"])

# -- Filter for June 2020 only --
june_data = df[(df["DateTime"] >= "2020-06-01") & (df["DateTime"] < "2020-07-01")].copy()

# -- Set time index and ensure hourly regularity --
june_data = june_data.set_index("DateTime")[["RH (%)"]]  # Keep only the RH column
june_data = june_data.resample("h").asfreq()  # hourly frequency

# -- Interpolate missing values --
june_data["RH (%)"] = june_data["RH (%)"].interpolate(method="time", limit_direction="both")

# -- Remove the last 2 hours of June --
june_data = june_data.iloc[:-2]

# -- Final check for row count --
assert len(june_data) == 718, f"Unexpected row count: {len(june_data)}"

# -- Save to CSV --
june_data.to_csv("/home/mpasillaspablo/CIMIS/RH_Files/70_june_RH_cleaned.csv")

# -- Confirmation --
print("Saved to /home/mpasillaspablo/CIMIS/RH_Files/70_june_RH_cleaned.csv")


Saved to /home/mpasillaspablo/CIMIS/RH_Files/70_june_RH_cleaned.csv
